# Step 1: Web scraping 

In [1]:
from requests import get
url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [2]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
#html_soup

In [3]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [4]:
first_movie = html_soup('div', class_ = 'lister-item mode-advanced')[0]
first_movie.h3.a.get_text()

'Logan'

In [5]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
first_year.get_text()

'(2017)'

In [6]:
first_rating = float(first_movie.strong.text)
first_rating

8.1

In [7]:
first_meta_score = first_movie.find('span', class_ = 'metascore favorable')
first_meta_score = float(first_meta_score.text)
first_meta_score

77.0

In [8]:
first_votes = first_movie.find('span', attrs = {'name':'nv'})
first_votes = float(first_votes['data-value'])
first_votes

610290.0

In [9]:
first_genre = first_movie.find('span', class_ = 'genre')
first_genre = first_genre.text
first_genre

'\nAction, Drama, Sci-Fi            '

In [10]:
first_Director_Stars = first_movie.find_all("p")
first_Director_Stars = first_Director_Stars[2]
first_Director_Stars = first_Director_Stars.find_all("a")
stars = []
for i in range(len(first_Director_Stars)):
    stars.append(first_Director_Stars[i].get_text())

stars

['James Mangold',
 'Hugh Jackman',
 'Patrick Stewart',
 'Dafne Keen',
 'Boyd Holbrook']

# Step 2: Feature engineering

In [11]:
# Lists to store the scraped data in

names = []
years = []
imdb_ratings = []
metascores = []
votes = []
genre = []
Dstars = []

# Extract data from individual movie container
for container in movie_containers:
# If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None:
        stars = []
# The name
        name = container.h3.a.text
        names.append(name)
# The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)
# The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
# The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))
# The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))
# Genres
        genres = container.find('span', class_ = 'genre').text
        genres = genres.replace("\n","")
        genre.append(genres)
# Director and stars
        first_Director_Stars = container.find_all("p")
        first_Director_Stars = first_Director_Stars[2]
        first_Director_Stars = first_Director_Stars.find_all("a")
        for i in range(len(first_Director_Stars)):
            stars.append(first_Director_Stars[i].get_text())
        Dstars.append(stars)
            

In [12]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
genre = []
Dstars = []

try:
    for i in range(0,347330,50): 
        url = 'https://www.imdb.com/search/title/?release_date=1990-01-01,2020-12-31&sort=num_votes,desc&start={}&ref_=adv_nxt'.format(i)
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
        for container in movie_containers:
            if container.find('div', class_ = 'ratings-metascore') is not None:
                stars = []
                #name of the movie
                name = container.h3.a.text
                names.append(name)

                #year of release
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                #rating it got
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                #metascore of the movie
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                #no. of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

                #genre
                genres = container.find('span', class_ = 'genre').text
                genres = genres.replace("\n","")
                genre.append(genres)

                #directors and actors
                first_Director_Stars = container.find_all("p")
                first_Director_Stars = first_Director_Stars[2]
                first_Director_Stars = first_Director_Stars.find_all("a")
                for i in range(len(first_Director_Stars)):
                    stars.append(first_Director_Stars[i].get_text())
                Dstars.append(stars)
except KeyboardInterrupt as e:
    print("Enough movies retrieved")
    

Enough movies retrieved


In [15]:
import pandas as pd
test_df = pd.DataFrame({'Movie': names,
'Year': years,
'Rating': imdb_ratings,
'Metascore': metascores,
'Votes': votes,
'Genre': genre,
'Director and stars':Dstars
})
test_df

,Movie,Year,Rating,Metascore,Votes,Genre,Director and stars
0,The Shawshank Redemption,(1994),9.3,80,2224204,Drama,"[Frank Darabont, Tim Robbins, Morgan Freeman, ..."
1,The Dark Knight,(2008),9.0,84,2198489,"Action, Crime, Drama","[Christopher Nolan, Christian Bale, Heath Ledg..."
2,Inception,(2010),8.8,74,1950317,"Action, Adventure, Sci-Fi","[Christopher Nolan, Leonardo DiCaprio, Joseph ..."
3,Fight Club,(1999),8.8,66,1772622,Drama,"[David Fincher, Brad Pitt, Edward Norton, Meat..."
4,Pulp Fiction,(1994),8.9,94,1744738,"Crime, Drama","[Quentin Tarantino, John Travolta, Uma Thurman..."
5,Forrest Gump,(1994),8.8,82,1715767,"Drama, Romance","[Robert Zemeckis, Tom Hanks, Robin Wright, Gar..."
6,The Matrix,(1999),8.7,73,1598086,"Action, Sci-Fi","[Lana Wachowski, Lilly Wachowski, Keanu Reeves..."
7,The Lord of the Rings: The Fellowship of the Ring,(2001),8.8,92,1588603,"Action, Adventure, Drama","[Peter Jackson, Elijah Wood, Ian McKellen, Orl..."
8,The Lord of the Rings: The Return of the King,(2003),8.9,94,1575863,"Adventure, Drama, Fantasy","[Peter Jackson, Elijah Wood, Viggo Mortensen, ..."
9,The Dark Knight Rises,(2012),8.4,78,1454434,"Action, Adventure","[Christopher Nolan, Christian Bale, Tom Hardy,..."


In [16]:
#Find the best movie among the bunch

test_df = pd.DataFrame({'Movie': names,
'Year': years,
'Rating': imdb_ratings,
'Metascore': metascores,
'Votes': votes,
'Genre': genre,
'Director and stars':Dstars,
})
test_df['Total'] = test_df["Rating"] + test_df["Metascore"] + test_df['Votes']
test_df.sort_values("Total",ascending=False)


,Movie,Year,Rating,Metascore,Votes,Genre,Director and stars,Total
0,The Shawshank Redemption,(1994),9.3,80,2224204,Drama,"[Frank Darabont, Tim Robbins, Morgan Freeman, ...",2224293.3
1,The Dark Knight,(2008),9.0,84,2198489,"Action, Crime, Drama","[Christopher Nolan, Christian Bale, Heath Ledg...",2198582.0
2,Inception,(2010),8.8,74,1950317,"Action, Adventure, Sci-Fi","[Christopher Nolan, Leonardo DiCaprio, Joseph ...",1950399.8
3,Fight Club,(1999),8.8,66,1772622,Drama,"[David Fincher, Brad Pitt, Edward Norton, Meat...",1772696.8
4,Pulp Fiction,(1994),8.9,94,1744738,"Crime, Drama","[Quentin Tarantino, John Travolta, Uma Thurman...",1744840.9
5,Forrest Gump,(1994),8.8,82,1715767,"Drama, Romance","[Robert Zemeckis, Tom Hanks, Robin Wright, Gar...",1715857.8
6,The Matrix,(1999),8.7,73,1598086,"Action, Sci-Fi","[Lana Wachowski, Lilly Wachowski, Keanu Reeves...",1598167.7
7,The Lord of the Rings: The Fellowship of the Ring,(2001),8.8,92,1588603,"Action, Adventure, Drama","[Peter Jackson, Elijah Wood, Ian McKellen, Orl...",1588703.8
8,The Lord of the Rings: The Return of the King,(2003),8.9,94,1575863,"Adventure, Drama, Fantasy","[Peter Jackson, Elijah Wood, Viggo Mortensen, ...",1575965.9
9,The Dark Knight Rises,(2012),8.4,78,1454434,"Action, Adventure","[Christopher Nolan, Christian Bale, Tom Hardy,...",1454520.4


# Step 3: Recommender system(Content based)

In [17]:

def listToString(s):  
    string = ""  
    for element in s:  
        string += element   
    return string  

test_df["Director and stars"] = test_df["Director and stars"].apply(listToString)

def combine_features(df):
    return df['Genre'] +" "+ df['Director and stars']

test_df["Combined_features"] = test_df.apply(combine_features,axis=1)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

text = test_df['Combined_features']#.astype(str)
cv = CountVectorizer()

count_matrix = cv.fit_transform(text)
print(count_matrix)


  (0, 17271)	1
  (0, 8063)	1
  (0, 6901)	1
  (0, 16703)	1
  (0, 4665)	1
  (0, 6843)	1
  (0, 5396)	1
  (1, 2834)	1
  (1, 5680)	1
  (1, 11562)	1
  (1, 1049)	1
  (1, 14564)	1
  (1, 3478)	1
  (1, 4187)	1
  (1, 69)	1
  (1, 5396)	1
  (2, 20626)	1
  (2, 14960)	1
  (2, 11851)	1
  (2, 7688)	1
  (2, 5125)	1
  (2, 14568)	1
  (2, 6398)	1
  (2, 17719)	1
  (2, 124)	1
  :	:
  (4759, 14702)	1
  (4759, 15196)	1
  (4759, 21322)	1
  (4759, 15769)	1
  (4759, 5857)	1
  (4759, 9846)	1
  (4759, 5396)	1
  (4760, 6444)	1
  (4760, 11889)	1
  (4760, 4100)	1
  (4760, 21563)	1
  (4760, 10297)	1
  (4760, 17107)	1
  (4760, 19680)	1
  (4760, 4187)	1
  (4760, 5396)	1
  (4761, 14571)	1
  (4761, 14865)	1
  (4761, 2299)	1
  (4761, 14263)	1
  (4761, 13189)	1
  (4761, 20319)	1
  (4761, 9300)	1
  (4761, 14196)	1
  (4761, 6146)	1


In [19]:
similarity_scores = cosine_similarity(count_matrix)
similarity_scores
similarity_scores[22]

array([0.11952286, 0.42163702, 0.28603878, ..., 0.1118034 , 0.10540926,
       0.10540926])

In [20]:
def get_title_from_index(index):
    return test_df["Movie"][index]

def get_index_from_title(title):
    return test_df[test_df.Movie == title].index.values[0]

In [21]:
get_title_from_index(22)

'The Prestige'

In [22]:
get_index_from_title("Thor")

76

In [24]:
# Content based recommender 
movie_user_likes = input("Enter the name of a movie you like: ")
print("Following are the recommendations for you:-")
movie_index = get_index_from_title(movie_user_likes)
similar_movies =  list(enumerate(similarity_scores[movie_index]))
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)
i=0
for element in sorted_similar_movies:
        print(get_title_from_index(element[0]))
        i=i+1
        if i>10:
            break

Enter the name of a movie you like: Fight Club
Following are the recommendations for you:-
Fight Club
The Curious Case of Benjamin Button
Naked Lunch
The Social Network
American Hustle
The Devil Wears Prada
A History of Violence
Hard Candy
Joy
Collateral Beauty
Animal Kingdom
